## Cleans TRACE Corporate Bond Data

In [1]:
# Imports
import pandas as pd
import numpy as np

In [2]:
# Loads in subset of prof Yoshio's data
with pd.read_csv("prof_yoshio_data/trace_credit_spreads.csv", chunksize=1000) as reader:
    data = reader.get_chunk()

In [3]:
data

,cusip_id,date,price,volume,ISSUE_ID,ISSUER_ID,PROSPECTUS_ISSUER_NAME,MATURITY,SECURITY_LEVEL,OFFERING_AMT,...,month1,accrued_interest,tau,age,rating,ytm,duration,tr_dirty_price,tr_ytm,cs
0,000325AA8,20020715,100.750000,18618000,36572,6006,AAF-MCQUAY INC,20030215,SEN,125000,...,20020815,3.677486,0.589041,6.432877,14,NaN,NaN,NaN,NaN,NaN
1,000325AA8,20020722,95.125000,235000,36572,6006,AAF-MCQUAY INC,20030215,SEN,125000,...,20020815,3.849102,0.569863,6.452055,14,NaN,NaN,NaN,NaN,NaN
2,000325AA8,20020724,100.675134,23250000,36572,6006,AAF-MCQUAY INC,20030215,SEN,125000,...,20020815,3.898135,0.564384,6.457534,14,NaN,NaN,NaN,NaN,NaN
3,000325AA8,20020726,100.625000,200000,36572,6006,AAF-MCQUAY INC,20030215,SEN,125000,...,20020815,3.947169,0.558904,6.463014,14,NaN,NaN,NaN,NaN,NaN
4,000325AA8,20020807,95.000000,416000,36572,6006,AAF-MCQUAY INC,20030215,SEN,125000,...,20020815,4.241367,0.526027,6.495890,14,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,00037BAA0,20150428,101.194000,60000,574887,43173,ABB FIN USA INC,20170508,SEN,500000,...,20150508,0.767610,2.030137,2.986301,6,1.028450,1.988149,102.836989,0.596744,0.431705
996,00037BAA0,20150429,101.054345,1450000,574887,43173,ABB FIN USA INC,20170508,SEN,500000,...,20150508,0.772099,2.027397,2.989041,6,1.097052,1.985332,102.821240,0.605305,0.491747
997,00037BAA0,20150430,101.122000,55000,574887,43173,ABB FIN USA INC,20170508,SEN,500000,...,20150508,0.776588,2.024658,2.991781,6,1.062663,1.982574,102.807227,0.613036,0.449627
998,00037BAA0,20150504,100.966402,2190000,574887,43173,ABB FIN USA INC,20170508,SEN,500000,...,20150508,0.794544,2.013699,3.002740,6,1.137560,1.971419,102.752543,0.643516,0.494044


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   cusip_id                1000 non-null   object 
 1   date                    1000 non-null   int64  
 2   price                   1000 non-null   float64
 3   volume                  1000 non-null   int64  
 4   ISSUE_ID                1000 non-null   int64  
 5   ISSUER_ID               1000 non-null   int64  
 6   PROSPECTUS_ISSUER_NAME  1000 non-null   object 
 7   MATURITY                1000 non-null   int64  
 8   SECURITY_LEVEL          1000 non-null   object 
 9   OFFERING_AMT            1000 non-null   int64  
 10  OFFERING_DATE           1000 non-null   int64  
 11  OFFERING_PRICE          852 non-null    float64
 12  INTEREST_FREQUENCY      1000 non-null   int64  
 13  COUPON                  1000 non-null   float64
 14  standard                1000 non-null   i

In [174]:
# data.memory_usage(deep=True) \
# .gt(8000)

As expected all "object" (ie. string) columns use up the majority of the memory. The rest (float/int64s) are using 8 bytes, however these may also be reduced to 16 or 32.

In [4]:
def cols(col_names, data_type):
    '''
    Takes in list of column names and a datatype and generates a dictionary
    '''
    return {col_names[i]: data_type for i in range(len(col_names))}

In [5]:
def clean_data_types(df):
    '''
    Cleans datatypes for raw data
    '''
    df.columns = map(str.lower, df.columns)
    
    # String/text columns
    string_cols = ['cusip_id', 'cusip', 'prospectus_issuer_name']
    strings = cols(string_cols, 'string[pyarrow]')

    bool_cols = ['callable', 'make_whole', 'conv', 'junior']
    bools = cols(bool_cols, 'bool')

    category_cols = ['security_level', 'rating_spr', 'rating_mdy', 'standard', 'putop', 'industry_code']
    categories = cols(category_cols, 'category')
    try:
        df = df.drop(["sp_rat", "moody_rat", "rating", 'rating_date_spr', 'rating_date_mdy'], axis=1)
    except: pass
        
    # Float/Integer columns
    int8_cols = ['interest_frequency']
    int8s = cols(int8_cols, 'int8')

    df['sic_code'] = df['sic_code'].fillna(0)
    int16_cols = ['sic_code']
    int16s = cols(int16_cols, 'int16')

    int32_cols = ['volume', 'offering_amt', 'amount_outstanding', 'issue_id', 'issuer_id']
    int32s = cols(int32_cols, 'int32')

    float32_cols = ['price', 'offering_price', 'coupon', 'accrued_interest', 'tau',
                   'age', 'ytm', 'duration', 'tr_dirty_price', 'tr_ytm', 'cs']
    float32s = cols(float32_cols, 'float32')

    # Dates
    # formula to convert to datetime: df['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
    df['offering_date'] = df['offering_date'].fillna(0)
    date_cols = ['date', 'maturity', 'offering_date', 'month1']
    dates = cols(date_cols, 'int32')

    # Applies type changes
    types = strings | bools | categories | int8s | int16s | int32s | float32s | dates

    return df.astype(types)

In [192]:
df = clean_data_types(df)
# df.memory_usage(deep=True)

In [198]:
df.info()
# Memory usage decreased by about half

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   cusip_id                1000 non-null   string  
 1   date                    1000 non-null   int32   
 2   price                   1000 non-null   float32 
 3   volume                  1000 non-null   int32   
 4   issue_id                1000 non-null   int32   
 5   issuer_id               1000 non-null   int32   
 6   prospectus_issuer_name  1000 non-null   string  
 7   maturity                1000 non-null   int32   
 8   security_level          1000 non-null   category
 9   offering_amt            1000 non-null   int32   
 10  offering_date           1000 non-null   int32   
 11  offering_price          852 non-null    float32 
 12  interest_frequency      1000 non-null   int8    
 13  coupon                  1000 non-null   float32 
 14  standard                1

In [204]:
# Loads in whole data
data = pd.read_csv("prof_yoshio_data/trace_credit_spreads.csv")

C:\Users\sroma\AppData\Local\Temp\ipykernel_25248\3165405631.py:2: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("prof_yoshio_data/trace_credit_spreads.csv")


14090424.827 KB


In [221]:
# Cleans data
cleaned_data = clean_data_types(data)

14090424.827 KB


In [ ]:
# Saves cleaned data
cleaned_data.to_csv('cleaned_data.csv')

In [668]:
# data.info()

In [667]:
# cleaned_data.info()

# Aggregates Data to Monthly

In [6]:
# Imports
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *

In [7]:
# Loads in subset of cleaned data
with pd.read_csv("cleaned_data.csv", chunksize=100_000, index_col=0) as reader:
    data = reader.get_chunk()

# csv doesn't preserve data types
data = clean_data_types(data)

In [8]:
print(f''' With the subset of data there are: 
    - {len(data['issuer_id'].unique())} unique issuers 
    - {len(data['issue_id'].unique())} unique bonds  
    - {len(data['date'].sort_values().unique())} days in the sample
    ''')

 With the subset of data there are: 
    - 43 unique issuers 
    - 192 unique bonds  
    - 4913 days in the sample
    


## Sample Exclusion Criteria
Based on Bai, Bali & Wen (2019):

1. Remove bonds that are not listed or traded in the US public market, which include bonds issued through private placement, bonds issued under the 144A rule, bonds that do not trade in US dollars, and bond issuers not in the jurisdiction of the United States.
2. Remove bonds that are structured notes, mortgage backed or asset backed, agency backed, or equity linked.
3. Remove convertible bonds since this option feature distorts the return calculation and makes it impossible to compare the returns of convertible and nonconvertible bonds.
4. Remove bonds that trade under 5 or above 1000. 
5. Remove bonds that have a floating coupon rate, which means the sample comprises only bonds with a fixed or zero coupon. This rule is applied based on the consideration of the accuracy in bond return calculation, given the challenge in tracking a floating-coupon bond’s cash flows.
6. Remove bonds that have less than one year to maturity. This rule is applied to all major corporate bond indices such as the Barclays Capital Corporate Bond Index, the Bank of America Merrill Lynch Corporate Master Index, and the Citi Fixed Income Indices. If a bond has less than one year to maturity, it will be delisted from major bond indices; hence, index-tracking investors will change their holding positions. This operation will distort the return calculation for bonds with less than one year to maturity; thus, we remove them from our sample.
7. For intraday data, we also eliminate bond transactions that are labeled as when-issued, locked-in, or have special sales conditions, and that have more than a twoday settlement.
8. Remove transaction records that are canceled and adjust records that are subsequently corrected or reversed.
9. Remove transaction records that have trading volume less than $10,000

### 3. Convertible Bonds

In [9]:
def filter_bond_convertible(data):
    '''
    Function which filters out convertible bonds.
    '''
    
    return data[data['conv'] == False]

In [10]:
data = filter_bond_convertible(data)

### 4. Bonds with irregular trading prices

In [11]:
def filter_bond_prices(data):
    '''
    Function which filters out bonds that trade below $5 or above $1000.
    '''
    
    return data[(data['price'].gt(5)) | (data['price'].lt(1000))]

In [12]:
data = filter_bond_prices(data)

### 6. Bonds with less than 1-year to maturity
Perhaps worthwhile to keep seperate as a robustness check?

In [13]:
def filter_bond_maturity(data):
    '''
    Function which filters out bonds with less than 1-year to maturity.
    Saves bonds with less than 1-year to maturity in a seperate file.
    '''
    data['date'] = pd.to_datetime(data['date'], format='%Y%m%d')
    data['maturity'] = pd.to_datetime(data['maturity'], format='%Y%m%d')

    data = data.sort_values(by='date')
    data['days_to_maturity'] = data['maturity'] - data['date']

    # Filters out bonds with less than 1-year to maturity
    data_one_year_bonds = data[data['days_to_maturity'].le('365 days')]
    data_one_year_bonds.to_csv('bonds_1_year_maturity.csv')
    
    return data[data['days_to_maturity'].gt('365 days')]

In [14]:
data = filter_bond_maturity(data)

### 9. Bonds with small trading volumes

In [15]:
def filter_bond_volume(data):
    '''
    Function which filters daily volumes of less than $10,000.
    '''
    
    return data[data['volume'].gt(10000)]

In [16]:
data = filter_bond_volume(data)

## Monthly Return Aggregation

The monthly corporate bond return at time $t$ is computed as 

<font size=5>
$
r_{i,t} = \frac{P_{i,t} + AI_{i,t} + C_{i,t}}{P_{i,t−1} + AI_{i,t−1}} − 1
$
</font>

Where $P_{i,t}$ is month $t$'s bond clean price, $AI_{i,t}$ is the accrued-interest, and $C_{i,t}$ is the coupon.

Excess returns are computed by deducting the 1-month Treasury bill rate.

Further, following Bai, Bali & Wen (2019), returns can be computed as either end of month t relative to end of month t-1, or end of month t to begining of month t. Where the end/beginning of the month is defined with a 5 day window. The date which is closest to the last/firsy trading day is used. If both such return calculations are available, then the end-to-end is prefered.

#### End-to-End Method

In [10]:
def end_month_returns(data):
    '''
    Function to compute monthly returns using the end-to-end method.
    '''
    # Ensures that data is sorted in chronological order
    data = data.sort_values(by='date')
    
    # Gets 5 days from the end of the month
    data['month_end'] = data['date'] + MonthEnd(0) - pd.Timedelta(days=5)
    
    # Groups by issue id and year-month: returns the last observation
    data = data.groupby(['issue_id', data['date'].dt.year, data['date'].dt.month]).tail(1).copy()
    
    # Creates the P + AI + C for time t
    data['full_price_t'] = data['price'] + data['accrued_interest'] + data['coupon']
    data['price_t'] = data['price'] + data['accrued_interest']
    
    # Lags P + AI
    data['price_t1'] = data.groupby('issue_id')['price_t'].shift(1)
    
    # Computes the return only if the transaction date is within 5 days of end of month
    data['return'] = np.where(~(data['date'] < data['month_end']), 
                               (data['full_price_t'] / data['price_t1']) - 1, np.nan)
    
    # Price return used for validation
    data['price_return'] = np.where(~(data['date'] < data['month_end']), 
                               (data['price'] / data.groupby('issue_id')['price'].shift(1)) - 1, np.nan)
    
    # Drops observations where there is no return, assigns month end value
    data = data.dropna(subset='return')
    data['date'] = data['date'] + MonthEnd(0)

    return data 

In [413]:
end_data = end_month_returns(data)

#### Start-to-End Method

In [11]:
def start_month_returns(data):
    '''
    Function to compute monthly returns using the start-to-end method.
    '''
    # Ensures that data is sorted in chronological order
    data = data.sort_values(by='date')
    
    # 5 days from start and end of month
    data['month_end'] = data['date'] + MonthEnd(0) - pd.Timedelta(days=5)
    data['month_start'] = data['date'] + MonthEnd(0) - MonthBegin(1) + pd.Timedelta(days=5)
    data['date_t'] = data['date'] + MonthEnd(0)
    
    # Seperate first and last trade dataframes
    first = data.groupby(['issue_id', data['date'].dt.year, data['date'].dt.month]).head(1).copy()
    last = data.groupby(['issue_id', data['date'].dt.year, data['date'].dt.month]).tail(1).copy()
    
    # Respective price only if date requirement is satisifed 
    first['price_t'] = np.where((first['date'] < first['month_start']),
                                 first['price'] + first['accrued_interest'], np.nan)
    last['full_price_t'] = np.where(~(last['date'] < last['month_end']),
                                      last['price'] + last['accrued_interest'] + last['coupon'], np.nan)
    
    # Merges first and last on true month end
    merged = pd.merge(first, last[['issue_id', 'date_t', 'full_price_t']], on=['issue_id', 'date_t'])

    # Computes the return
    merged['return'] = (merged['full_price_t'] / merged['price_t']) - 1
    
    # Drops observations where there is no return, assigns month end value
    data = merged.dropna(subset='return').copy()
    data['date'] = data['date'] + MonthEnd(0)
    
    return data

In [562]:
start_data = start_month_returns(data)

#### Merging of Returns

In [12]:
def merge_start_end(start_data, end_data):
    '''
    Function to merge start and end method return calculations.
    Preference for end method.
    '''
    cols = ['date', 'issue_id', 'cusip_id', 'volume', 'issuer_id',
       'prospectus_issuer_name', 'maturity', 'offering_amt',
       'interest_frequency', 'cusip', 'industry_code', 'sic_code', 'rating_spr', 'rating_mdy',
       'amount_outstanding', 'tau', 'age', 'ytm',
       'duration', 'tr_dirty_price', 'tr_ytm', 'return']

    # Subsets data and outer merges
    end_data = end_data[cols]
    start_data = start_data[cols]

    merged = pd.merge(end_data, start_data, on=['issue_id', 'date'], how='outer', suffixes=('','_start'))

    # Takes data from end method, if not available takes from start method
    for col in cols[2:]:
        merged[col] = np.where(merged[col].isna(), merged[col+'_start'], merged[col])
    
    merged = merged[cols]
    
    return merged

In [663]:
data_merged = merge_start_end(start_data, end_data)

## Complete Dataset

In [13]:
def agg_to_monthly(data):
    '''
    Function to perform all sample exclusion, and monthly aggregation functions 
    '''
    
    # Filtering functions
    data = (data.pipe(filter_bond_convertible)
            .pipe(filter_bond_prices)
            .pipe(filter_bond_maturity)
            .pipe(filter_bond_volume)
        )
    
    # Return calculation
    end_data = end_month_returns(data)
    start_data = start_month_returns(data)
    
    # Final merge
    data_merged = merge_start_end(start_data, end_data)
    
    return data_merged

In [4]:
# Loads in data
data = pd.read_csv("cleaned_data.csv", index_col=0)
data = clean_data_types(data)

In [697]:
# data['issue_id'].unique().shape

(2107,)

In [14]:
# Performs monthly aggregation
monthly_data = agg_to_monthly(data)

In [1]:
monthly_data

NameError: name 'monthly_data' is not defined

In [698]:
# monthly_data['issue_id'].unique().shape

(1895,)

In [700]:
monthly_data.shape

(87369, 22)

In [699]:
# monthly_data.to_csv('monthly_data.csv')